In [1]:
import os
import PIL
import h5py
import math
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.utils.np_utils import to_categorical
%matplotlib inline

Using TensorFlow backend.


In [2]:
OUT_FILE = 'data/aug_train_data.h5'
METADATA = 'metadata.npy'
DATA_FOLDER = 'data/augmentated'
SHAPE = (150, 250, 3)
BATCH_SIZE = 16

In [3]:
labels = os.listdir(DATA_FOLDER)
labels = [l for l in labels if ".DS_Store" not in l]
labels_path = [os.path.join(DATA_FOLDER, l) for l in labels]
print(labels_path)
n_labels = len(labels)
n_images = np.sum([len(os.listdir(l)) for l in labels_path])

label2idx = {j:to_categorical(i, n_labels)[0] for i,j in enumerate(labels)}
np.save(METADATA, label2idx)

model = VGG16(include_top=False, input_shape=SHAPE)
model.layers.pop()
    
print('model output shape: {}', model.output_shape)

f = h5py.File(OUT_FILE)
output_shape = (n_images, *model.output_shape[1:])
chunk_shape = (BATCH_SIZE, *model.output_shape[1:])
try:
    f.pop('x')
    f.pop('y')
except:
    None

img_data = f.create_dataset('x', shape=output_shape,
                            dtype=np.float32, chunks=chunk_shape)
labels_data = f.create_dataset('y', shape=(n_images, n_labels),
                               dtype=np.float32, chunks=(BATCH_SIZE, n_labels))

['data/augmentated/ALB', 'data/augmentated/BET', 'data/augmentated/DOL', 'data/augmentated/LAG', 'data/augmentated/NoF', 'data/augmentated/OTHER', 'data/augmentated/SHARK', 'data/augmentated/YFT']


model output shape: {} (None, 4, 7, 512)


In [4]:
from collections import deque
from time import time

t0 = time()
ts = deque(maxlen=5)
idxx = 0

for path, label in zip(labels_path, labels):
    files = os.listdir(path)
    files = [f for f in files if ".DS_Store" not in f]
    n_files = len(files)
    next_idxx = idxx + n_files
    labels_data[idxx:next_idxx] = label2idx[label]
    idx = 0
    for i in range(math.ceil(n_files / BATCH_SIZE)):
        tt0 = time()
        eta = np.mean(ts) * (n_images - idxx - idx) if len(ts) != 0 else 999
        print('%s: %04d/%04d (ETA: %.2fs)...' %
              (label, idx, n_files, eta), end='\r')
        next_idx = min(n_files, idx + BATCH_SIZE)
        batch_n = next_idx - idx
        batch_x = np.ndarray((batch_n, *SHAPE), dtype=np.float32)
        for j in range(batch_n):
            img = PIL.Image.open(os.path.join(path, files[idx + j]))
            img = img.resize(SHAPE[:2][::-1])
            img = np.asarray(img, dtype=np.float32)
            batch_x[j] = img
    
        batch_x = preprocess_input(batch_x)
        img_data[idxx + idx:idxx + idx + batch_n] = model.predict(batch_x)
        idx = next_idx
        ts.append((time() - tt0) / batch_n)
    
    idxx = next_idxx
    print(' '*50, end='\r')
    print('{}: Done'.format(label))
    
print('Done ({})'.format((time() - t0)))

ALB: Done


BET: Done


DOL: Done


LAG: Done


NoF: Done


OTHER: Done


SHARK: Done


YFT: Done
Done (6059.247812986374)


In [5]:
f.close()

In [5]:
OUT_FILE = 'data/test_data.h5'
METADATA = 'metadata.npy'
DATA_FOLDER = 'data/test'
SHAPE = (150, 250, 3)
BATCH_SIZE = 16

In [6]:
labels = os.listdir(DATA_FOLDER)
labels = [l for l in labels if ".DS_Store" not in l]
labels_path = [os.path.join(DATA_FOLDER, l) for l in labels]
print(labels_path)
n_labels = len(labels)
n_images = np.sum([len(os.listdir(l)) for l in labels_path])

label2idx = {j:to_categorical(i, n_labels)[0] for i,j in enumerate(labels)}
np.save(METADATA, label2idx)

model = VGG16(include_top=False, input_shape=SHAPE)
model.layers.pop()
    
print('model output shape: {}', model.output_shape)

f = h5py.File(OUT_FILE)
output_shape = (n_images, *model.output_shape[1:])
chunk_shape = (BATCH_SIZE, *model.output_shape[1:])
try:
    f.pop('x')
    f.pop('y')
except:
    None

img_data = f.create_dataset('x', shape=output_shape,
                            dtype=np.float32, chunks=chunk_shape)
labels_data = f.create_dataset('y', shape=(n_images, n_labels),
                               dtype=np.float32, chunks=(BATCH_SIZE, n_labels))

['data/test/ALB', 'data/test/BET', 'data/test/DOL', 'data/test/LAG', 'data/test/NoF', 'data/test/OTHER', 'data/test/SHARK', 'data/test/YFT']


model output shape: {} (None, 4, 7, 512)


In [7]:
from collections import deque
from time import time

t0 = time()
ts = deque(maxlen=5)
idxx = 0

for path, label in zip(labels_path, labels):
    files = os.listdir(path)
    files = [f for f in files if ".DS_Store" not in f]
    n_files = len(files)
    next_idxx = idxx + n_files
    labels_data[idxx:next_idxx] = label2idx[label]
    idx = 0
    for i in range(math.ceil(n_files / BATCH_SIZE)):
        tt0 = time()
        eta = np.mean(ts) * (n_images - idxx - idx) if len(ts) != 0 else 999
        print('%s: %04d/%04d (ETA: %.2fs)...' %
              (label, idx, n_files, eta), end='\r')
        next_idx = min(n_files, idx + BATCH_SIZE)
        batch_n = next_idx - idx
        batch_x = np.ndarray((batch_n, *SHAPE), dtype=np.float32)
        for j in range(batch_n):
            img = PIL.Image.open(os.path.join(path, files[idx + j]))
            img = img.resize(SHAPE[:2][::-1])
            img = np.asarray(img, dtype=np.float32)
            batch_x[j] = img
    
        batch_x = preprocess_input(batch_x)
        img_data[idxx + idx:idxx + idx + batch_n] = model.predict(batch_x)
        idx = next_idx
        ts.append((time() - tt0) / batch_n)
    
    idxx = next_idxx
    print(' '*50, end='\r')
    print('{}: Done', label)
    
print('Done ({})', (time() - t0))

{}: Done ALB


{}: Done BET


{}: Done DOL


{}: Done LAG


{}: Done NoF


{}: Done OTHER


{}: Done SHARK


{}: Done YFT
Done ({}) 237.97780179977417


In [8]:
f.close()